In [18]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
from collections import defaultdict

In [19]:
bzcat = pd.read_excel('../data/bzcat_full.xlsx')

Here the data are collected directly into a Pandas dataframe. This is because the FIRST, 2MASS, and WISE data may appear in the response or not. A 'defaultdict' construction might work faster, but the code will be more bulky with it. The runtime is of second concern here as we anyway have limitations in requests per minute (time sleep = 1s)

In [ ]:
result_df = pd.DataFrame(columns=['Source name', 'SDSS id', 'u', 'u_err', 'g',
    'g_err', 'r', 'r_err', 'i', 'i_err', 'z', 'z_err', 'extinction_r', 'J', 
    'H', 'K', 'w1', 'w2', 'w3', 'w4', 'FIRST'])

j, n = 0, bzcat.shape[0]
bzcat.reset_index(inplace=True)

for index, obj in bzcat.iterrows():
    j += 1
    print(f'Processing {j} out of {n}')
    result_dict = {'Source name': obj['BZCAT5 Source name']}
    
    url = 'http://skyserver.sdss.org/dr17/en/tools/explore/Summary.aspx?ra=' \
        + obj['RA (J2000.0)'] + '&dec=' + obj['Dec (J2000.0)']
    response = requests.get(url)
    page = BeautifulSoup(response.text, 'html.parser') 
    tds = page.find_all('td')
    
    if len(tds) > 0:
        result_dict.update({
            'SDSS id': tds[6].text, 'u': tds[11].text, 'g': tds[12].text,
            'r': tds[13].text, 'i': tds[14].text, 'z': tds[15].text, 'u_err':
            tds[16].text, 'g_err': tds[17].text, 'r_err': tds[18].text, 
            'i_err': tds[19].text, 'z_err': tds[20].text, 'extinction_r': 
            tds[26].text
        })  
        for i in range(31, len(tds)):
            if tds[i].text == 'FIRST':
                result_dict.update({'FIRST': tds[i+1].text})
            if tds[i].text == '2MASS':
                result_dict.update({'J': tds[i+1].text, 'H': tds[i+2].text,
                                    'K': tds[i+3].text})
            if tds[i].text == 'WISE':
                result_dict.update({'w1': tds[i+1].text, 'w2': tds[i+2].text,
                                    'w3': tds[i+3].text, 'w4': tds[i+4].text})
    
    result_df = pd.concat(
        [result_df, pd.DataFrame(result_dict, index=[0])], ignore_index=True)
    time.sleep(1)   
   
result_df.to_csv('../data/SDSS_data.xlsx')